# Setup

In [69]:
from optibook.synchronous_client import Exchange

import time
import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


# Connecting to the Exchange

In [98]:
exchange = Exchange()
_ = exchange.connect()

2023-04-05 14:18:18,355 [asyncio   ] [MainThread  ] Using selector: EpollSelector


# Getting market information

In [71]:
# Load all instruments (products) available on the exchange
instruments = exchange.get_instruments()
instruments

{'PHILIPS_B': Instrument(instrument_id=PHILIPS_B, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, paused=False),
 'PHILIPS_A': Instrument(instrument_id=PHILIPS_A, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, paused=False)}

In [90]:
# Define a variable storing the instrument_id of the product we are interested in
instrument_id = 'PHILIPS_A'

### Order Book

In [78]:
# Load current limit order book for the instrument
exchange.get_last_price_book(instrument_id)

PriceBook(PHILIPS_A 2023-04-05 13:05:08.509762)
 #bids | price | #asks 
       |  99.9 | 10000 
       |  94.4 |  250  
       |  93.8 |  410  
   10  |  93.7 |       
  250  |  93.6 |       
  400  |  93.2 |       
 10000 |  87.6 |       

### Public tradeticks

In [79]:
# Load and store a list of all public tradeticks since the instantiation of the Exchange (upto a max limit)
# These are trades between any participant, you may or may not have participated in these
trade_tick_history = exchange.get_trade_tick_history(instrument_id)

# Display last 5
trade_tick_history[-5:]

[TradeTick(timestamp=2023-04-05 13:04:52.017961, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8680),
 TradeTick(timestamp=2023-04-05 13:04:52.054917, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8682),
 TradeTick(timestamp=2023-04-05 13:04:52.073978, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8684),
 TradeTick(timestamp=2023-04-05 13:04:54.972794, instrument_id=PHILIPS_A, price=93.80000000000001, volume=10, aggressor_side=bid, buyer=, seller=, trade_id=8688),
 TradeTick(timestamp=2023-04-05 13:05:08.541397, instrument_id=PHILIPS_A, price=93.80000000000001, volume=5, aggressor_side=bid, buyer=, seller=, trade_id=8697)]

In [80]:
# Poll all public tradeticks since the last time this method was called
exchange.poll_new_trade_ticks(instrument_id)

[TradeTick(timestamp=2023-04-05 13:04:45.982588, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8634),
 TradeTick(timestamp=2023-04-05 13:04:46.060540, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8636),
 TradeTick(timestamp=2023-04-05 13:04:46.136236, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8638),
 TradeTick(timestamp=2023-04-05 13:04:46.206755, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8640),
 TradeTick(timestamp=2023-04-05 13:04:46.306551, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8642),
 TradeTick(timestamp=2023-04-05 13:04:46.370461, instrument_id=PHILIPS_A, price=93.80000000000001, volume=20, aggressor_side=bid, buyer=, seller=, trade_id=8644),
 TradeTick(timestamp=2

### Private trades

In [81]:
# Load and store a list of all private trades you participated in since the instantiation of the Exchange (upto a max limit)
trade_history = exchange.get_trade_history('PHILIPS_A')

# Display last 5
trade_history[-5:]

[]

In [82]:
# Poll all private trades since the last time this method was called
exchange.poll_new_trades(instrument_id)

[]

2023-04-05 13:05:34,853 [client    ] [Thread-8    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


### Position & PnL

In [11]:
# Load current positions in all instruments
exchange.get_positions()

{'DEMO': 0, 'TEST': 0, 'MKT1': 0, 'MKT2': 0, 'MKT3': 0}

In [12]:
# Load current positions in all instruments, including cash component (how much did we invest or gain by trading each instrument)
exchange.get_positions_and_cash()

{'DEMO': {'volume': 0, 'cash': 0.0},
 'TEST': {'volume': 0, 'cash': 0.0},
 'MKT1': {'volume': 0, 'cash': 0.0},
 'MKT2': {'volume': 0, 'cash': 0.0},
 'MKT3': {'volume': 0, 'cash': 0.0}}

In [13]:
# Current PnL: sum of all cash components and value of the positions (evaluated against last-traded price) 
exchange.get_pnl()

0.0

# Orders

### Inserting

In [31]:
# Insert bid LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
# The returned value is a 'InsertOrderResponse' which, if successfull, contains the order_id of the order
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='limit')

InsertOrderResponse(success=True, order_id=104326, error_reason='None')

In [32]:
# Insert ask LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='limit')

InsertOrderResponse(success=True, order_id=104327, error_reason='None')

In [33]:
# Insert bid IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='ioc')

InsertOrderResponse(success=True, order_id=104330, error_reason='None')

In [ ]:
# Insert ask IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='ioc')

### Deleting & amending

In [30]:
# Load a list of own outstanding/resting orders
exchange.get_outstanding_orders(instrument_id)

{104295: OrderStatus(order_id=104295, instrument_id=DEMO, price=10.0, volume=5, side=bid)}

In [ ]:
# Delete an order by order_id, the returned DeleteOrderRequest contains a boolean showing whether the delete was succesful
exchange.delete_order(instrument_id, order_id=1234)

In [ ]:
# Delete all outstanding orders for an instrument
exchange.delete_orders(instrument_id)

In [ ]:
# Change (amend) the volume of an existing resting order by order_id, the returned AmendOrderResponse contains a boolean showing whether the amend was succesful
exchange.amend_order(instrument_id, order_id=1234, volume=30)

# Code Snippets

Combining a few of the exchange interactions above, we can write a code snippet to aggressively trade out of all currently held positions using IOC orders. 

That means selling all products you are long, and buying back all you are short. 

This is useful for starting with a clean slate, but doing so will come at a potentially large cost. 

You are not guaranteed to end up with a zero position afterwards, only that volume which is also available to trade in the market will be bought/sold.

In [99]:
MIN_SELLING_PRICE = 0.10
MAX_BUYING_PRICE = 100000.00

positions = exchange.get_positions()
pnl = exchange.get_pnl()

print(f'Positions before: {positions}')
print(f'\nPnL before: {pnl:.2f}')

print(f'\nTrading out of positions')
for iid, pos in positions.items():
    if pos > 0:
        print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
        exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
    elif pos < 0:
        print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
        exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
    else:
        print(f'-- No initial position in {iid}, skipping..')
    
    time.sleep(0.10)

time.sleep(1.0)

positions = exchange.get_positions()
pnl = exchange.get_pnl()
print(f'\nPositions after: {positions}')
print(f'\nPnL after: {pnl:.2f}')

Positions before: {'PHILIPS_A': -1, 'PHILIPS_B': 0}

PnL before: 43.80

Trading out of positions
-- Inserting buy order for 1 lots of PHILIPS_A, with limit price 100000.00
-- No initial position in PHILIPS_B, skipping..

Positions after: {'PHILIPS_A': 0, 'PHILIPS_B': 0}

PnL after: 43.80


2023-04-05 14:19:28,171 [client    ] [Thread-15   ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.
